#**Data Preprocessing Tools**


---



##**1. Importation de donnees**

In [19]:
import pandas as pd

#importer le dataset
dataset_path='Life Expectancy Data.csv'
df=pd.read_csv(dataset_path)
print('le dataset a ete importe avec success')

le dataset a ete importe avec success


##**2.0 Nettoyage des donnees**

In [20]:
#identifier et supprimer les colonnes qui contient une seule valeur


#afficher les nombre de valeurs unique pour chaque colonne
print(df.nunique())

#identifier les colonne avec 1 valeur unique et les supprimer
columns_to_drop = df.columns[df.nunique() == 1]
print("\n les colonnes avec 1 seul valeur unique sont : ",columns_to_drop)

df = df.drop(columns=columns_to_drop)
print("\n les colonnes avec 1 seul valeur unique ont ete supprimee")

Country                             193
Year                                 16
Status                                2
Life expectancy                     362
Adult Mortality                     425
infant deaths                       209
Alcohol                            1076
percentage expenditure             2328
Hepatitis B                          87
Measles                             958
 BMI                                608
under-five deaths                   252
Polio                                73
Total expenditure                   818
Diphtheria                           81
 HIV/AIDS                           200
GDP                                2490
Population                         2278
 thinness  1-19 years               200
 thinness 5-9 years                 207
Income composition of resources     625
Schooling                           173
dtype: int64

 les colonnes avec 1 seul valeur unique sont :  Index([], dtype='object')

 les colonnes avec 1 seul valeu

In [23]:
#identifier et supprimer les colonnes qui ont tres peu de valeurs (variance)
seuil_variance = 0.01


#calcul et affichage de la variance pour chaque colonne (numerical columns)
print("------------------------------------------------------------------------------------------")
print("------------------------------------------------------------------------------------------")
print("the data types of each column is : \n",df.dtypes)
df_numerical = df.select_dtypes(include=['number'])
print("------------------------------------------------------------------------------------------")
print("------------------------------------------------------------------------------------------")
print("the variance of each column is : \n",df_numerical.var())

#supprimer les colones avec des variance inferieur que le seuil
print("------------------------------------------------------------------------------------------")
print("------------------------------------------------------------------------------------------")
columns_to_drop = df_numerical.columns[df_numerical.var() < seuil_variance]
print("les colonnes avec une variance < ",seuil_variance," sont : ",columns_to_drop )
df = df.drop(columns=columns_to_drop)
print("les colonnes avec une variance < ",seuil_variance," ont ete supprimee")


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
the data types of each column is : 
 Country                             object
Year                                 int64
Status                              object
Life expectancy                    float64
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                              int64
 BMI                               float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP                                float64
Population                         float64
 thinne

In [24]:
#identifier et supprimer les lignes qui sont des doublons


#afficher les doublons
print("les doublons sont : \n",df[df.duplicated()])

#supprimer les doublons
df = df.drop_duplicates()
print("les doublons ont ete supprimee")

les doublons sont : 
 Empty DataFrame
Columns: [Country, Year, Status, Life expectancy , Adult Mortality, infant deaths, Alcohol, percentage expenditure, Hepatitis B, Measles ,  BMI , under-five deaths , Polio, Total expenditure, Diphtheria ,  HIV/AIDS, GDP, Population,  thinness  1-19 years,  thinness 5-9 years, Income composition of resources, Schooling]
Index: []

[0 rows x 22 columns]
les doublons ont ete supprimee


In [28]:
#La gestion des outliers avec IQR ( optionel et depend du model a utiliser apres pour l'entrainement)

'''
df_numerical = df.select_dtypes(include=['number'])
#calculate interquartile range
Q1 = df_numerical.quantile(0.25)
Q3 = df_numerical.quantile(0.75)
IQR = Q3 - Q1
print("------------------------------------------------------------------------------------------")
print("------------------------------------------------------------------------------------------")
print("the IQR of each column is : \n",IQR)


#calculate the outlier cutoff
cut_off = IQR * 1.5
print("------------------------------------------------------------------------------------------")
print("------------------------------------------------------------------------------------------")
print("the outlier cutoff is : \n",cut_off)
lower, upper = Q1 - cut_off, Q3 + cut_off


#identify the outliers
outliers = [x for x in df_numerical if x < lower[x] or x > upper[x]]
print("------------------------------------------------------------------------------------------")
print("------------------------------------------------------------------------------------------")
print("the outliers are : \n",outliers)
'''


'\ndf_numerical = df.select_dtypes(include=[\'number\'])\n#calculate interquartile range\nQ1 = df_numerical.quantile(0.25)\nQ3 = df_numerical.quantile(0.75)\nIQR = Q3 - Q1\nprint("------------------------------------------------------------------------------------------")\nprint("------------------------------------------------------------------------------------------")\nprint("the IQR of each column is : \n",IQR)\n\n\n#calculate the outlier cutoff\ncut_off = IQR * 1.5\nprint("------------------------------------------------------------------------------------------")\nprint("------------------------------------------------------------------------------------------")\nprint("the outlier cutoff is : \n",cut_off)\nlower, upper = Q1 - cut_off, Q3 + cut_off\n\n\n#identify the outliers\noutliers = [x for x in df_numerical if x < lower[x] or x > upper[x]]\nprint("------------------------------------------------------------------------------------------")\nprint("----------------------------

In [36]:
# la gestion des valeurs manquantes

import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer

# Separate numerical and categorical columns
df_numerical = df.select_dtypes(include=[np.number])
df_categorical = df.select_dtypes(exclude=[np.number])

# Loop through columns
for col in df.columns:
    missing_percent = df[col].isnull().mean() * 100

    if missing_percent < 5:
        if col in df_numerical.columns:
            imputer = SimpleImputer(strategy='median')
            df[[col]] = imputer.fit_transform(df[[col]])
        else:
            imputer = SimpleImputer(strategy='most_frequent')
            df[[col]] = imputer.fit_transform(df[[col]])

    elif 5 <= missing_percent <= 30:
        if col in df_numerical.columns:
            # Using KNN imputer for numerical values
            knn_imputer = KNNImputer(n_neighbors=5)
            df[[col]] = knn_imputer.fit_transform(df[[col]])
        else:
            # Use most_frequent as a placeholder for categorical regression
            # or consider more advanced imputation techniques (like using RandomForest or logistic regression)
            imputer = SimpleImputer(strategy='most_frequent')
            df[[col]] = imputer.fit_transform(df[[col]])

    else:
        df.drop(columns=col, inplace=True)

print("missing values have been dealt with")

missing values have been dealt with


In [39]:
#encoding
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer

# Separate numerical and categorical columns
df_numerical = df.select_dtypes(include=[np.number])
df_categorical = df.select_dtypes(exclude=[np.number])


#for categorical features if unique values <5 use one hot encoding else use label encoder
for col in df_categorical.columns:
  if df[col].nunique() < 5:
    #one hot encoding
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [col])], remainder='passthrough')
    df = pd.DataFrame(ct.fit_transform(df))
  else:
    #label encoding
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])



In [40]:
#scaling

from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
df=pd.DataFrame(ss.fit_transform(df))



In [41]:
#save the cleaned dataframe

df.to_csv('cleaned_data.csv', index=False)
print("the cleaned data has been saved")

the cleaned data has been saved


#**enhanced preprocessing with logs through out preprocessing**

In [42]:
# -*- coding: utf-8 -*-
"""
Enhanced Data Preprocessing Template with Comprehensive Reporting
"""

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from datetime import datetime
import os

class PreprocessingReporter:
    def __init__(self, dataset_name):
        self.report_lines = []
        self.dataset_name = dataset_name
        self.start_time = datetime.now()

        # Initialize report
        self.add_header()

    def add_header(self):
        """Add report header with timestamp and dataset info"""
        self.report_lines.extend([
            "="*80,
            "DATA PREPROCESSING REPORT",
            "="*80,
            f"Dataset: {self.dataset_name}",
            f"Processing Date: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}",
            f"Generated by: Enhanced Preprocessing Template",
            "="*80,
            ""
        ])

    def add_section(self, title):
        """Add a new section to the report"""
        self.report_lines.extend([
            f"\n{'-'*60}",
            f"{title.upper()}",
            f"{'-'*60}\n"
        ])

    def add_content(self, content):
        """Add content to the report"""
        if isinstance(content, list):
            self.report_lines.extend(content)
        else:
            self.report_lines.append(str(content))

    def add_dataframe_summary(self, df, title):
        """Add dataframe summary statistics"""
        self.add_content([
            f"\n{title}:",
            f"Shape: {df.shape}",
            f"Columns: {list(df.columns)}",
            f"Data Types:\n{df.dtypes.to_string()}",
            f"\nMissing Values Summary:",
            f"{df.isnull().sum().to_string()}",
            f"\nDescriptive Statistics:",
            f"{df.describe().to_string()}",
            ""
        ])

    def save_report(self, filename="preprocessing_report.txt"):
        """Save the report to a text file"""
        # Add completion timestamp
        end_time = datetime.now()
        processing_time = end_time - self.start_time

        self.report_lines.extend([
            "\n" + "="*80,
            "PREPROCESSING COMPLETED",
            "="*80,
            f"End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}",
            f"Total Processing Time: {processing_time}",
            f"Report saved as: {filename}",
            "="*80
        ])

        with open(filename, 'w', encoding='utf-8') as f:
            f.write('\n'.join(self.report_lines))

        print(f"Preprocessing report saved as: {filename}")

def enhanced_preprocessing_pipeline(dataset_path, output_filename="cleaned_data.csv", report_filename="preprocessing_report.txt"):
    """
    Enhanced preprocessing pipeline with comprehensive reporting
    """
    # Initialize reporter
    dataset_name = os.path.basename(dataset_path)
    reporter = PreprocessingReporter(dataset_name)

    # Step 1: Data Import
    reporter.add_section("1. DATA IMPORT")
    try:
        df_original = pd.read_csv(dataset_path)
        reporter.add_content([
            f"✓ Dataset imported successfully from: {dataset_path}",
            f"Original dataset shape: {df_original.shape}",
            ""
        ])

        # Add initial data summary
        reporter.add_dataframe_summary(df_original, "ORIGINAL DATASET SUMMARY")

    except Exception as e:
        reporter.add_content(f"✗ Error importing dataset: {e}")
        return None

    # Create a copy for processing
    df = df_original.copy()

    # Step 2: Remove Single-Value Columns
    reporter.add_section("2. SINGLE-VALUE COLUMN REMOVAL")

    unique_counts = df.nunique()
    columns_to_drop_single = df.columns[unique_counts == 1].tolist()

    if columns_to_drop_single:
        df = df.drop(columns=columns_to_drop_single)
        reporter.add_content([
            f"Decision: Remove columns with only 1 unique value",
            f"Reason: These columns provide no information for analysis",
            f"Columns removed: {columns_to_drop_single}",
            f"Columns dropped: {len(columns_to_drop_single)}",
            f"Remaining columns: {df.shape[1]}",
            ""
        ])
    else:
        reporter.add_content([
            "✓ No single-value columns found",
            "Decision: No action needed",
            ""
        ])

    # Step 3: Low Variance Column Removal
    reporter.add_section("3. LOW VARIANCE COLUMN REMOVAL")

    variance_threshold = 0.01
    df_numerical = df.select_dtypes(include=['number'])

    if not df_numerical.empty:
        variances = df_numerical.var()
        low_variance_cols = variances[variances < variance_threshold].index.tolist()

        reporter.add_content([
            f"Variance threshold: {variance_threshold}",
            f"Numerical columns analyzed: {len(df_numerical.columns)}",
            f"Variance summary:\n{variances.to_string()}",
            ""
        ])

        if low_variance_cols:
            df = df.drop(columns=low_variance_cols)
            reporter.add_content([
                f"Decision: Remove low variance columns",
                f"Reason: Low variance columns may not contribute significantly to analysis",
                f"Columns removed: {low_variance_cols}",
                f"Columns dropped: {len(low_variance_cols)}",
                f"Remaining columns: {df.shape[1]}",
                ""
            ])
        else:
            reporter.add_content([
                "✓ No low variance columns found",
                "Decision: No action needed",
                ""
            ])
    else:
        reporter.add_content([
            "No numerical columns found for variance analysis",
            "Decision: Skip variance-based removal",
            ""
        ])

    # Step 4: Duplicate Removal
    reporter.add_section("4. DUPLICATE ROW REMOVAL")

    duplicate_count = df.duplicated().sum()

    if duplicate_count > 0:
        df = df.drop_duplicates()
        reporter.add_content([
            f"Decision: Remove duplicate rows",
            f"Reason: Duplicate rows can bias analysis and model training",
            f"Duplicate rows found: {duplicate_count}",
            f"Rows after removal: {df.shape[0]}",
            ""
        ])
    else:
        reporter.add_content([
            "✓ No duplicate rows found",
            "Decision: No action needed",
            ""
        ])

    # Step 5: Missing Value Handling
    reporter.add_section("5. MISSING VALUE HANDLING")

    missing_summary = df.isnull().sum()
    missing_percentages = (df.isnull().sum() / len(df)) * 100

    reporter.add_content([
        "Missing value strategy:",
        "• <5% missing: Simple imputation (median for numerical, mode for categorical)",
        "• 5-30% missing: Advanced imputation (KNN for numerical, mode for categorical)",
        "• >30% missing: Drop column",
        "",
        "Missing value analysis:"
    ])

    for col in df.columns:
        missing_count = missing_summary[col]
        missing_percent = missing_percentages[col]

        if missing_count > 0:
            reporter.add_content(f"  {col}: {missing_count} missing ({missing_percent:.2f}%)")

    reporter.add_content("")

    # Separate numerical and categorical columns
    df_numerical = df.select_dtypes(include=[np.number])
    df_categorical = df.select_dtypes(exclude=[np.number])

    columns_dropped_missing = []
    columns_imputed = []

    for col in df.columns:
        missing_percent = (df[col].isnull().sum() / len(df)) * 100

        if missing_percent > 30:
            # Drop column
            df.drop(columns=col, inplace=True)
            columns_dropped_missing.append(col)
            reporter.add_content(f"  {col}: DROPPED (>{30}% missing)")

        elif missing_percent > 0:
            # Impute
            if col in df_numerical.columns:
                if missing_percent < 5:
                    imputer = SimpleImputer(strategy='median')
                    method = "median imputation"
                else:
                    imputer = KNNImputer(n_neighbors=5)
                    method = "KNN imputation"
            else:
                imputer = SimpleImputer(strategy='most_frequent')
                method = "mode imputation"

            df[[col]] = imputer.fit_transform(df[[col]])
            columns_imputed.append((col, method))
            reporter.add_content(f"  {col}: IMPUTED using {method}")

    reporter.add_content([
        "",
        f"Summary:",
        f"  Columns dropped due to missing values: {len(columns_dropped_missing)}",
        f"  Columns imputed: {len(columns_imputed)}",
        f"  Remaining columns: {df.shape[1]}",
        ""
    ])

    # Step 6: Categorical Encoding
    reporter.add_section("6. CATEGORICAL ENCODING")

    df_categorical = df.select_dtypes(exclude=[np.number])

    if not df_categorical.empty:
        reporter.add_content([
            "Encoding strategy:",
            "• <5 unique values: One-Hot Encoding",
            "• ≥5 unique values: Label Encoding",
            "",
            "Categorical columns analysis:"
        ])

        encoding_actions = []

        for col in df_categorical.columns:
            unique_count = df[col].nunique()

            if unique_count < 5:
                # One-hot encoding
                ct = ColumnTransformer(
                    transformers=[('encoder', OneHotEncoder(drop='first', sparse_output=False), [col])],
                    remainder='passthrough'
                )
                df_encoded = ct.fit_transform(df)

                # Get feature names
                feature_names = ct.named_transformers_['encoder'].get_feature_names_out([col]).tolist()
                feature_names.extend([c for c in df.columns if c != col])

                df = pd.DataFrame(df_encoded, columns=feature_names)
                encoding_actions.append(f"  {col}: ONE-HOT ENCODED ({unique_count} unique values)")

            else:
                # Label encoding
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
                encoding_actions.append(f"  {col}: LABEL ENCODED ({unique_count} unique values)")

        reporter.add_content(encoding_actions)
        reporter.add_content([
            "",
            f"Final shape after encoding: {df.shape}",
            ""
        ])

    else:
        reporter.add_content([
            "✓ No categorical columns found",
            "Decision: No encoding needed",
            ""
        ])

    # Step 7: Feature Scaling
    reporter.add_section("7. FEATURE SCALING")

    reporter.add_content([
        "Decision: Apply StandardScaler to all features",
        "Reason: Ensures all features have mean=0 and std=1 for better model performance",
        ""
    ])

    # Get column names before scaling
    column_names = df.columns.tolist()

    # Apply scaling
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    df = pd.DataFrame(df_scaled, columns=column_names)

    reporter.add_content([
        f"✓ StandardScaler applied to all {df.shape[1]} features",
        f"Final dataset shape: {df.shape}",
        ""
    ])

    # Step 8: Final Summary
    reporter.add_section("8. FINAL DATASET SUMMARY")
    reporter.add_dataframe_summary(df, "CLEANED DATASET SUMMARY")

    # Step 9: Save Results
    reporter.add_section("9. SAVING RESULTS")

    try:
        df.to_csv(output_filename, index=False)
        reporter.add_content([
            f"✓ Cleaned dataset saved as: {output_filename}",
            f"Final shape: {df.shape}",
            ""
        ])
    except Exception as e:
        reporter.add_content(f"✗ Error saving cleaned dataset: {e}")

    # Save report
    reporter.save_report(report_filename)

    return df

# Main execution
if __name__ == "__main__":
    # Configuration
    dataset_path = 'Life Expectancy Data.csv'
    output_file = 'cleaned_data.csv'
    report_file = 'preprocessing_report.txt'

    # Run preprocessing pipeline
    print("Starting enhanced preprocessing pipeline...")
    print("="*50)

    cleaned_data = enhanced_preprocessing_pipeline(
        dataset_path=dataset_path,
        output_filename=output_file,
        report_filename=report_file
    )

    if cleaned_data is not None:
        print(f"\n✓ Preprocessing completed successfully!")
        print(f"✓ Cleaned data saved as: {output_file}")
        print(f"✓ Detailed report saved as: {report_file}")
        print(f"✓ Final dataset shape: {cleaned_data.shape}")
    else:
        print("\n✗ Preprocessing failed. Check the report for details.")

Starting enhanced preprocessing pipeline...
Preprocessing report saved as: preprocessing_report.txt

✓ Preprocessing completed successfully!
✓ Cleaned data saved as: cleaned_data.csv
✓ Detailed report saved as: preprocessing_report.txt
✓ Final dataset shape: (2938, 22)
